# Experimentation

Let's load the data and configs from the previous notebook first.

In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext memory_profiler

import pickle
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
import requests
from IPython.display import display

import azs_helpers.l2r_helper as azs
from azs_helpers.azure_search_client import azure_search_client as azs_client 

interim_data_dir = Path.cwd() / 'data' / 'interim'
df = pd.read_csv(interim_data_dir / 'normalized_features.csv')

display(df.head(5))

## Split dataset into features and judgments

The machine learning libraries we're using expect us to separate our training data into features $X$ and judgment values $y$. Let's do that real quick: 


In [ ]:
X, y = df.drop(columns=['grade', 'query_id'], axis=1), df.grade

### Hold out a portion of the data for testing.

We need to hold out part of the data to validate that our model works.

For this tutorial, we will hold out **10%** of the data using `scikit-learn.model_selection.GroupShuffleSplit`. This means that the model will use **90%** of the queries for training & validation, while **10%** of the queries (that the model has never seen before) will be used to evaluate the final model's performance.

#### Query labels for documents

Our data preparation and model training process must be aware of query-document relationships in our dataset for accurate training results, so we'll need to pass in the query labels (`query_ids`) as well. The reason this is important is because the loss function used for optimization depends on the NDCG. As discussed above, the NDCG is a metric which indicates how close is an ordered list to its ideal order. To achieve this, the model needs to be aware of how to create those lists out of the training set, which can be achieved by grouping the data using the query_id.

Read the following references for more information:

- [Scikit-Learn: Cross Validation](https://scikit-learn.org/stable/modules/cross_validation.html)
- [Scikit-Learn: GroupShuffleSplit](https://scikit-learn.org/stable/modules/cross_validation.html#group-shuffle-split)

In [ ]:
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

# Hold out 10% of our queries in the test set.
random_seed = 42 # we are using a static random seed so we can easily reproduce results. The actual number has no meaning.
gss = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=random_seed)

# Query labels for each document in our dataset.
query_ids = df['query_id'].to_numpy()

train_index, test_index = next(gss.split(X, y, groups=query_ids))
print(f"Number of samples in train: {len(train_index)}")
print(f"Number of samples in test: {len(test_index)}")

# Split our features, judgment values, and query labels into train & test sets.
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]
query_ids_train, query_ids_test = query_ids[train_index], query_ids[test_index]

# Model Setup

Now that we've completed our feature engineering steps, we can move on to running our experiment. We'll be using the open-source [XGBoost library](https://xgboost.readthedocs.io/en/latest/index.html), which includes a few implementations of [**Pairwise**](https://en.wikipedia.org/wiki/Learning_to_rank#Pairwise_approach) and [**Listwise**](https://en.wikipedia.org/wiki/Learning_to_rank#Listwise_approach) ranking algorithms.

By setting `'objective': 'rank:ndcg'` in the model constructor, we've chosen to use the [LambdaMART](https://www.microsoft.com/en-us/research/publication/from-ranknet-to-lambdarank-to-lambdamart-an-overview/) algorithm to maximize **listwise** [NDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain).

Another objective function that can be used for this problem is `rank:pairwise`, which will perform **pairwise** training instead of listwise training.

### Further Reading:
- [Learning to Rank: From Pairwise Approach to Listwise Approach](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-2007-40.pdf)
- XGBoost Documentation links:
    - [XGBRanker Class](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRanker)
    - [XGBoost Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html)

In [ ]:
import xgboost as xgb

params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 8, 'n_estimators': 500}

ranker = xgb.XGBRanker(**params)
ranker

### Train with K-Fold Cross Validation

K-Fold cross validation is a robust way to avoid overfitting during model training. Let's train with K-fold cross validation with `K=9`. We'll be using `scikit-learn`'s [GroupKFold class](https://scikit-learn.org/stable/modules/cross_validation.html#group-k-fold) to split our training data into 9 approximately even-sized portions. `GroupKFold` ensures that all documents for a particular query remain within the same fold. Before we can get started, we'll need to augment our query labels for training.
- If you're running this on Binder, set`n_splits` = 2 instead.

#### Computing Document counts per Query

During the training phase, we need to transform our query labels to be compatible with XGBoost. The `GroupKFold` cross-validation tool provided by `scikit-learn` expects a group (or query) label for every document in the dataset. We've already provided this in the form of `query_ids`.

Let's take a look at the example below. Given a simple dataset of 3 queries:

- Query 1: [doc1, doc2, doc3]
- Query 2: [doc4]
- Query 3: [doc5, doc6]

**`scikit-learn` expects the following format**:

```python
# query_ids = [query1, query1, query1, query2, query3, query3]
query_ids = [1, 1, 1, 2, 3, 3]
```

**`xgboost` expects the following format**:

```python
query_doc_counts = [3, 1, 2]
```
    
`xgboost` expects document counts for each query (as opposed to a query label for every document), sorted by query. We can take our scikit-learn compatible query labels and aggregate them using numpy, as follows:

```python
labels, query_doc_counts = np.unique(query_ids, return_counts=True)
```

or, alternatively:
```python
query_doc_counts = np.unique(query_group_labels, return_counts=True)[1]
```

For reference, please refer to the [XGBoost Docs](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRanker)

### Calculating our Results
Calling `ranker.predict(X_test)` here will allow us to get all of the results. Since `xgb_scores` is a flat list of scores that aren't grouped by query, we'll need to slice it with sliding windows to create predicted rankings for each query. Feel free to read the docstring of `azs_helpers.l2r_helper.compute_judgments_after_training` for more information.

We expect uneven group sizes like the following:

- Query 1: 10 search results
- Query 2: 8 search results
- Query 3: 7 search results

In [ ]:
%%time
from sklearn.model_selection import GroupKFold

def calculate_ndcg_for_experiment(ranker, X, y, query_ids):
    n_splits = 9
    group_kfold = GroupKFold(n_splits)

    ndcg_results = []

    for train_idx, validation_idx in group_kfold.split(X, y, groups=query_ids):
        X_train_fold, X_validation_fold = X.iloc[train_idx], X.iloc[validation_idx]
        y_train_fold, y_validation_fold = y.iloc[train_idx], y.iloc[validation_idx]
        
        # XGBoost expects query counts instead of query labels.
        # https://stackoverflow.com/questions/10741346/numpy-most-efficient-frequency-counts-for-unique-values-in-an-array
        train_groups_xgb = np.unique(query_ids[train_idx], return_counts=True)[1]
        validation_groups_xgb = np.unique(query_ids[validation_idx], return_counts=True)[1]
        
        ranker.fit(X_train_fold, y_train_fold, train_groups_xgb,
                   eval_set=[(X_validation_fold, y_validation_fold)], eval_group=[validation_groups_xgb],
                   eval_metric='ndcg',
                   verbose=False)
        xgb_scores = ranker.predict(X_validation_fold)
        model_judgments, azs_judgments, baseline = azs.compute_judgments_after_training(xgb_scores, validation_groups_xgb, X_validation_fold, y_validation_fold)
        ndcg_results_for_fold = azs.evaluate_ndcg(k_start=1, k_end=10, azs=azs_judgments, predicted=model_judgments)
        ndcg_results.append(ndcg_results_for_fold)

    return ndcg_results

# %memit ndcg_results = calculate_ndcg_for_experiment(ranker, X_train, y_train, query_ids_train)
ndcg_results = calculate_ndcg_for_experiment(ranker, X_train, y_train, query_ids_train)

### Visualize Cross-Validation Results

Now that we've trained our model multiple times, we can visualize the training results.

Datapoints represent the mean across all training runs, while vertical bars are the standard deviation.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

azs.show_ndcg_results(ndcg_results, 1, 10)

### Putting it all together

Now that we're confident that our model performs better than Azure Search's built-in retrieval algorithm, let's run it against the test set! This time around, we'll be using all of the training data available to us.

In [ ]:
train_groups_xgb = np.unique(query_ids_train, return_counts=True)[1]
test_groups_xgb = np.unique(query_ids_test, return_counts=True)[1]

ranker.fit(X_train, y_train, train_groups_xgb,
           eval_metric='ndcg',
           verbose=False)

xgb_scores = ranker.predict(X_test)
model_judgments, azs_judgments, baseline = azs.compute_judgments_after_training(xgb_scores, test_groups_xgb, X_test, y_test)

test_ndcg_results = azs.evaluate_ndcg(k_start=1, k_end=10, plot=True, show_lift=True, azs=azs_judgments, predicted=model_judgments)

### Feature Importances
We can view which features ended up being the most important in our trained model:

In [ ]:
feature_importances = dict(zip(X_train.columns, ranker.feature_importances_))
feature_importances_sorted = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse=True)}
feature_importances_sorted

### How to re-rank a search request's results using our trained model
When re-ranking an Azure Search query, we need to go through the same process to transform the results into features as we did for our training data. 

This means

1. Get features from Azure Search
2. Compute any additional features that was used int raining
3. Apply the same level of normalization 

Once we get the features in the expected format, we can feed them into the trained model to infer a new order, which will be used to re-rank our results.

In [ ]:
def rerank_search_query(service, query, model, expected_features):
    expected_features_format = pd.DataFrame(columns = expected_features, dtype='float64')
    
    json_search_results = azs.get_search_results_from_service(service, query, [])
    search_results = pd.json_normalize(json_search_results).fillna(0)
    search_results["query"] = query

    search_features = azs.customize_features(search_results.drop(columns=['url_en_us'], axis=1))
    normalized_search_features = azs.normalize_features(expected_features_format.append(search_features))
    
    prediction_score = model.predict(normalized_search_features.drop(columns=['query_id'], axis=1, errors='ignore'))
    search_results['prediction'] = prediction_score

    new_df = pd.DataFrame()
    new_df['Base Azure Search order'] = search_results.apply(lambda row: "{0}\n{1}".format(row['title_en_us'], row['url_en_us']), axis=1) 
    new_df['Trained model order'] = search_results.sort_values('prediction', ascending=False).apply(lambda row: "{0}\n{1}".format(row['title_en_us'], row['url_en_us']), axis=1).tolist() #['title_en_us'].tolist()

    return new_df
service_metadata_config_path = Path.cwd() / 'config' / 'config.json'
azs_service = azs_client.from_json(service_metadata_config_path)
azs.pretty_print(rerank_search_query(azs_service, 'Windows Server 2019', ranker, X.columns))